In [1]:
import torch
import pandas as pd
import pyperclip
import os.path as osp

pth = "output/evaluate/{}/all-results.th"
pth2 = "output/evaluate/{}/{}-results.th"

In [2]:
FULL_PRIN = True
row_order = ['AMES', 'CYP2C19_Veith', 'Tox21->NR-AR']
CLS_TRIC = 'auc'


# functions help you see results
def parse_result(result, keep_first=True, return_best_epoch=False):
    rs = []
    last_epoch = -1
    best_epoch = None
    for r in result:
        epoch = r['checkpoint']['epoch']

        if epoch <= last_epoch:
            if keep_first:
                break
            else:
                rs = []

        last_epoch = epoch

        res = r['result']
        rob = {}
        rob['epoch'] = epoch
        todo = res.keys() if FULL_PRIN else row_order
        for dataset in todo:
            rob[dataset] = res[dataset][CLS_TRIC]
        rs.append(rob)
        if return_best_epoch:
            best_epoch = r['checkpoint'].get('best_epoch')
    rs = pd.DataFrame(rs)
    if return_best_epoch:
        return rs, best_epoch
    else:
        return rs

In [26]:
def batch_best_load(todos):
    head = None
    contant = ""
    for do in todos:
        exp = do[:do.rfind('-')]
        if not osp.exists(pth2.format(do, "cls")):
            contant += f"{exp}\t{do}\n"
            continue
        result = torch.load(pth2.format(do, "cls"))

        df, best_epoch = parse_result(result, return_best_epoch=True, keep_first=False)
        df = df[df['epoch'] == best_epoch]

        assert len(df) > 0, f"{do} best epoch not exist, {best_epoch}: {parse_result(result)}"

        df.drop(columns=['epoch'], inplace=True)
        df['mean'] = df.mean(axis=1)
        a = df.iloc[0]
        head_tmp = 'exp\ttag\t' + "\t".join(a.keys())

        if head is None:
            contant = head_tmp + '\n' + contant
            head = head_tmp
        if head != head_tmp:
            contant += "\n" + head_tmp + '\n'
            head = head_tmp

        contant += f"{exp}\t{do}\t" + "\t".join(a.values.astype(str)) + '\n'

    pyperclip.copy(contant)

todos = [f'{o}-{i}' for o in
         [
             'allseed-35ds-scaf-60ex-moce-dotSAG-ESLoss',
             'allseed-35ds-scaf-60ex-moce-dotSAG',
             'allseed-35ds-scaf-60ex-moce-ESLoss',
             'allseed-35ds-scaf-60ex-moe',
             'allseed-35ds-scaf-gnn',
         ]
         for i in range(10)]

batch_best_load(todos)